In [ ]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)
from llama_index.experimental.query_engine.pandas import (
    PandasInstructionParser,
)
from llama_index.core import PromptTemplate
from llama_index.llms.ollama import Ollama
import pandas as pd
import textwrap

In [ ]:
def formatar_texto(texto):
  texto_formatado = textwrap.fill(texto, width=100)
  return texto_formatado

In [ ]:
df = pd.read_csv("vendas.csv")

In [ ]:
df.head()

In [ ]:
def descricao_colunas(df):
  descricao = '\n'.join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
  return 'Aqui estão os detalhes das colunas do DataFrame:\n' + descricao


In [ ]:
# llm = Ollama(
#     model="llama3.2:3b", 
#     request_timeout=120.0, 
#     context_window=10240, 
#     temperature=0.05,
#     max_tokens=10240,
#     top_k=5,
# )

# llm = Ollama(
#     model="deepseek-r1:1.5b", 
#     request_timeout=120.0, 
#     context_window=10240,
#     max_tokens=10240,
# )

# llm = Ollama(
#     model="gemma3:4b", 
#     request_timeout=120.0, 
#     context_window=10240,
#     max_tokens=10240,
# )

# llm = Ollama(
#     model="codegemma:2b", 
#     request_timeout=120.0, 
#     context_window=10240,
#     max_tokens=10240,
# )

# llm = Ollama(
#     model="phi", 
#     request_timeout=120.0,
# )
# llm = Ollama(
#     model="gemma3:1b", 
#     request_timeout=120.0,
# )

llm = Ollama(
    model="qwen2.5-coder:1.5b", 
    request_timeout=120.0,
    temperature=0.5,
    top_k=5,
)

In [ ]:
def pipeline_consulta(df, llm, descricao_colunas):

    instruction_str = (
        "1. Converta a Consulta para um código Python executável usando a biblioteca Pandas.\n"
        "2. A linha de código final deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
        "3. O código deve ser um código Python válido e representar uma solução para a consulta.\n"
        "4. IMPRIMA SOMENTE A EXPRESSÃO.\n"
        "5. NÃO coloque a EXPRESSÃO entre aspas.\n"
        "6. Dica: caso necessário lemebre-se que o Pandas permite que vocë faça `groupby` por mais de uma coluna do dataframe ao mesmo tempo.\n"
    )

    pandas_prompt_str = (
        "Você é uma Analista de Dados Sênior e está trabalhando com um dataframe Pandas em Python chamado `df`.\n"
        "O dataframe contém dados de vendas, incluindo as seguintes colunas:\n"
        "{colunas_detalhes}\n\n"
        "Este é o resultado de `print(df.head())`:\n"
        "{df_str}\n\n"
        "Siga as seguintes instruções:\n"
        "{instruction_str}\n"
        "Consulta: {query_str}\n\n"
        "Expressão:"
    )
    response_synthesis_prompt_str = (
        "Você é um Analista de dados Sênior, dado uma pergunta de entrada elabore uma resposta para a consulta a partir do resultado da saída do Pandas.\n"
        "Responda de forma natural, a resposta deve focar na consulta.\n"
        "Você NÃO deve inventar dados, caso a saída do Pandas seja um erro, peça desculpa solicite que a consulta seja realizada novamente.\n" 
        "Você NÃO deve utilizar introduções como 'A resposta é:' ou algo semelhante.\n"
        "Consulta: {query_str}\n\n"
        "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
        "Saída do Pandas: {pandas_output}\n\n"
        "Resposta:"
        "Ao final, você deve informar de forma explicativa o resultado, e também o código usado para gerar a resposta, no formato: O código utilizado foi {pandas_instructions}\n"
    )

    pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
        instruction_str=instruction_str, df_str=df.head(5), colunas_detalhes=descricao_colunas
    )
    pandas_output_parser = PandasInstructionParser(df)
    response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

    qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
    )

    qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
    qp.add_links(
        [
            Link("input", "response_synthesis_prompt", dest_key="query_str"),
            Link(
                "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
            ),
            Link(
                "pandas_output_parser",
                "response_synthesis_prompt",
                dest_key="pandas_output",
            ),
        ]
    )

    qp.add_link("response_synthesis_prompt", "llm2")

    return qp

In [ ]:
qp = pipeline_consulta(df, llm, descricao_colunas(df))

response = qp.run(
    query_str="Em quais cidades temos filiais?",
)

In [ ]:
print(response.message.content)

In [ ]:
qp = pipeline_consulta(df, llm, descricao_colunas(df))

response = qp.run(
    query_str="Qual é o tipo de produto com maior quantidade na filial B?",
)

In [ ]:
print(response.message.content)